[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/nn_adv/class_03/Laborat%C3%B3rios/lab_utils_ml_adv_week_3.zip
      
!unzip -n -q lab_utils_ml_adv_week_3.zip

In [ ]:
# Testar se estamos no Google Colab
# Necessário para ativar widgets
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Avaliação e Seleção de Modelos

Quantificar o desempenho de um algoritmo de aprendizado e comparar diferentes modelos são algumas das tarefas comuns ao aplicar o aprendizado de máquina ao mundo real.

Neste laboratório, você praticará essas tarefas usando as dicas compartilhadas em sala de aula. Especificamente, você irá:

* dividir os conjuntos de dados em conjuntos de treinamento, validação cruzada e teste
* avaliar modelos de regressão e classificação
* adicionar recursos polinomiais para melhorar o desempenho de um modelo de regressão linear
* comparar várias arquiteturas de redes neurais

Este laboratório também o ajudará a se familiarizar com o código que você verá na atividade avaliativa do tópico "Boas Práticas durante o Treino e Avaliação de Redes Neurais". Vamos começar!

## Importações e configuração do laboratório

Primeiro, você importará os pacotes necessários para as tarefas deste laboratório. Também foram incluídos alguns comandos para tornar os resultados mais legíveis posteriormente, reduzindo a verbosidade e suprimindo avisos não críticos.

In [ ]:
# para cálculos de matrizes e carregamento de dados
import numpy as np

# para criar modelos de regressão linear e preparar dados
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# para criar e treinar redes neurais
import tensorflow as tf

# funções personalizadas
import utils

# Reduzir a precisão da exibição em matrizes numéricas
np.set_printoptions(precision=2)

# suprimir avisos
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

## Regressão

Primeiro, você terá a tarefa de desenvolver um modelo para um problema de regressão. Você recebeu o conjunto de dados abaixo, que consiste em 50 exemplos de um recurso de entrada `x` e seu alvo correspondente `y`.

In [ ]:
# Carregar o conjunto de dados do arquivo de texto
data = np.loadtxt('./data/data_w3_ex1.csv', delimiter=',')

# Dividir as entradas e saídas em matrizes separadas
x = data[:,0]
y = data[:,1]

# Converta matrizes 1-D em 2-D porque os comandos posteriores exigirão isso
x = np.expand_dims(x, axis=1)
y = np.expand_dims(y, axis=1)

print(f"o formato da entrada x é: {x.shape}")
print(f"o formato dos alvos y é: {y.shape}")

Você pode plotar o conjunto de dados para ter uma ideia de como o alvo se comporta em relação à entrada. Caso queira inspecionar o código, você pode encontrar a função `plot_dataset()` no arquivo `utils.py` fora deste notebook.

In [ ]:
# Plotar todo o conjunto de dados
utils.plot_dataset(x=x, y=y, title="entrada vs. alvo")

## Dividir o conjunto de dados em conjuntos de treinamento, validação cruzada e teste

Em laboratórios anteriores, você pode ter usado todo o conjunto de dados para treinar seus modelos. Na prática, entretanto, é melhor reter uma parte dos dados para medir a capacidade de generalização do modelo para novos exemplos. Isso permitirá que você saiba se o modelo se ajustou demais ao conjunto de treinamento (_overfitting_).

Conforme mencionado na aula, é comum dividir seus dados em três partes:

* ***conjunto de treinamento*** - usado para treinar o modelo
* ***conjunto de validação cruzada (também chamado de validação, desenvolvimento ou conjunto dev)*** - usado para avaliar as diferentes configurações de modelo que você está escolhendo. Por exemplo, você pode usá-lo para tomar uma decisão sobre quais recursos polinomiais devem ser adicionados ao conjunto de dados.
* ***conjunto de teste*** - usado para fornecer uma estimativa justa do desempenho do modelo escolhido em relação a novos exemplos. Isso não deve ser usado para tomar decisões enquanto você ainda estiver desenvolvendo os modelos.

O Scikit-learn fornece uma função [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) para dividir seus dados nas partes mencionadas acima. Na célula de código abaixo, você dividirá o conjunto de dados inteiro em 60% de treinamento, 20% de validação cruzada e 20% de teste.

In [ ]:
# Obtenha 60% do conjunto de dados como o conjunto de treinamento.
# Coloque os 40% restantes em variáveis temporárias: x_ e y_.
x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

# Dividir o subconjunto de 40% acima em dois:
# uma metade para validação cruzada e a outra para o conjunto de teste
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Excluir variáveis temporárias
del x_, y_

print(f"o formato do conjunto de treino (entrada) é: {x_train.shape}")
print(f"o formato do conjunto de treino (alvo) é: {y_train.shape}\n")
print(f"o formato do conjunto de validação cruzada (entrada) é: {x_cv.shape}")
print(f"o formato do conjunto de validação cruzada (alvo) é: {y_cv.shape}\n")
print(f"o formato do conjunto de teste (entrada) é: {x_test.shape}")
print(f"o formato do conjunto de teste (alvo) é: {y_test.shape}")

Você pode plotar o conjunto de dados novamente abaixo para ver quais pontos foram usados como dados de treinamento, validação cruzada ou teste.

In [ ]:
utils.plot_train_cv_test(x_train, y_train, x_cv, y_cv, x_test, y_test, title="input vs. target")

## Ajustar um modelo linear

Agora que você dividiu os dados, uma das primeiras coisas que pode tentar é ajustar um modelo linear. Isso será feito nas próximas seções abaixo.

### Feature scaling
Anteriormente você viu que geralmente é uma boa ideia realizar o _feature scaling_ para ajudar o modelo a convergir mais rapidamente. Isso é especialmente verdadeiro se os recursos de entrada tiverem faixas de valores muito diferentes. 

Mais adiante neste laboratório, você adicionará termos polinomiais para que os recursos de entrada tenham, de fato, intervalos diferentes. Por exemplo, $x$ varia de aproximadamente 1600 a 3600, enquanto $x^2$ varia de 2,56 milhões a 12,96 milhões. 

Você usará apenas $x$ para esse primeiro modelo, mas é bom praticar o dimensionamento de recursos agora para poder aplicá-lo mais tarde. Para isso, você usará a classe [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) do scikit-learn. Ele calcula o _z-score_ de suas entradas.

Para relembrar, o _z-score_ é dado pela _equação:
_

$$ z = \frac{x - \mu}{\sigma} $$

em que $\mu$ é a média dos valores dos recursos e $\sigma$ é o desvio padrão.

O código abaixo mostra como preparar o conjunto de treinamento usando a classe mencionada. Você pode plotar os resultados novamente para inspecionar se eles ainda seguem o mesmo padrão de antes. O novo gráfico deve ter um intervalo reduzido de valores para `x`.

In [ ]:
# Inicializar a classe
scaler_linear = StandardScaler()

# Calcule a média e o desvio padrão do conjunto de treinamento e, em seguida, transforme-o
X_train_scaled = scaler_linear.fit_transform(x_train)

print(f"Média computada do conjunto de treinamento: {scaler_linear.mean_.squeeze():.2f}")
print(f"Desvio padrão calculado do conjunto de treinamento: {scaler_linear.scale_.squeeze():.2f}")

# Plot the results
utils.plot_dataset(x=X_train_scaled, y=y_train, title="entradas escaladas vs. alvo")

### Treinar o modelo

Em seguida, você criará e treinará um modelo de regressão. Para este laboratório, você usará a classe [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), mas observe que há outros [linear regressors](https://scikit-learn.org/stable/modules/classes.html#classical-linear-regressors) que também podem ser usados.

In [ ]:
# Inicializar a classe
linear_model = LinearRegression()

# Treinar o modelo
linear_model.fit(X_train_scaled, y_train )

### Avaliar o modelo

Para avaliar o desempenho do seu modelo, você medirá o erro dos conjuntos de treinamento e validação cruzada. Para o erro de treinamento, lembre-se da equação para calcular o erro quadrático médio (MSE):

$$J_{train}(\vec{w}, b) = \frac{1}{2m_{train}}\left[\sum_{i=1}^{m_{train}}(f_{\vec{w},b}(\vec{x}_{train}^{(i)}) - y_{train}^{(i)})^2\right]$$

O Scikit-learn também tem uma função interna [`mean_squared_error()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) que você pode usar. Observe, porém, que [de acordo com a documentação](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error), a implementação do scikit-learn divide apenas por `m` e não por `2*m`, em que `m` é o número de exemplos. Conforme mencionado anteriormente(aulas sobre a função de custo), a divisão por `2m` é uma convenção que seguiremos, mas os cálculos ainda devem funcionar independentemente de você incluí-la ou não. Portanto, para corresponder à equação acima, você pode usar a função scikit-learn e dividir por 2, conforme mostrado abaixo. Também foi incluída uma implementação de loop-for para que você possa verificar se é igual. 

Outro aspecto a ser observado: como você treinou o modelo em valores escalonados (ou seja, usando o _z-score_), você também deve alimentar o conjunto de treinamento escalonado em vez de seus valores brutos.

In [ ]:
# Alimente o conjunto de treinamento escalado e obtenha as previsões
yhat = linear_model.predict(X_train_scaled)

# Use a função do scikit-learn e divida por 2
print(f"MSE de treino (usando a função do sklearn): {mean_squared_error(y_train, yhat) / 2}")

# Implementação do for-loop
total_squared_error = 0

for i in range(len(yhat)):
    squared_error_i  = (yhat[i] - y_train[i])**2
    total_squared_error += squared_error_i                                              

mse = total_squared_error / (2*len(yhat))

print(f"treinamento MSE (implementação loop-for): {mse.squeeze()}")

Em seguida, você pode calcular o MSE para o conjunto de validação cruzada basicamente com a mesma equação:

$$J_{cv}(\vec{w}, b) = \frac{1}{2m_{cv}}\left[\sum_{i=1}^{m_{cv}}(f_{\vec{w},b}(\vec{x}_{cv}^{(i)}) - y_{cv}^{(i)})^2\right]$$

Assim como no conjunto de treinamento, você também deverá escalar o conjunto de validação cruzada. Um aspecto *importante* a ser observado ao usar o _z-score_ é que você deve usar a média e o desvio padrão do **conjunto de treinamento** ao escalar o conjunto de validação cruzada. Isso serve para garantir que seus recursos de entrada sejam transformados conforme o esperado pelo modelo. Uma maneira de obter intuição é com este cenário:

* Digamos que seu conjunto de treinamento tenha um recurso de entrada igual a "500", que é reduzido para "0,5" usando o _z-score_.
* Após o treinamento, seu modelo é capaz de mapear com precisão essa entrada escalonada `x=0,5` para a saída de destino `y=300`. 
* Agora, digamos que você tenha implantado esse modelo e um dos seus usuários tenha fornecido uma amostra igual a 500. 
* Se você obtiver o z-score dessa amostra de entrada usando qualquer outro valor da média e do desvio padrão, ele poderá não ser escalado para `0,5` e seu modelo provavelmente fará uma previsão errada (ou seja, não será igual a `y=300`). 



Você dimensionará o conjunto de validação cruzada abaixo usando o mesmo `StandardScaler` usado anteriormente, mas apenas chamando o método [`transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.transform) em vez de [`fit_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.fit_transform).

In [ ]:
# Escale o conjunto de validação cruzada usando a média e o desvio padrão do conjunto de treinamento
X_cv_scaled = scaler_linear.transform(x_cv)

print(f"Média usada para escalar o conjunto de VC: {scaler_linear.mean_.squeeze():.2f}")
print(f"Desvio padrão usado para dimensionar o conjunto de VC: {scaler_linear.scale_.squeeze():.2f}")

# Alimentar o conjunto de validação cruzada escalonada
yhat = linear_model.predict(X_cv_scaled)

# Use a função de utilidade do scikit-learn e divida por 2
print(f"MSE de validação cruzada: {mean_squared_error(y_cv, yhat) / 2}")

## Adição de recursos polinomiais

Nos gráficos anteriores, você deve ter notado que o alvo `y` aumenta mais acentuadamente em valores menores de `x` em comparação com os maiores. Uma linha reta pode não ser a melhor escolha porque o alvo `y` parece se achatar à medida que `x` aumenta. Agora que você tem esses valores de MSE de treinamento e validação cruzada do modelo linear, pode tentar adicionar recursos polinomiais para ver se consegue obter um desempenho melhor. O código será basicamente o mesmo, mas com algumas etapas extras de pré-processamento. Vamos ver isso a seguir.

### Criar os recursos adicionais

Primeiro, você gerará os recursos polinomiais do seu conjunto de treinamento. O código abaixo demonstra como fazer isso usando a classe [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). Ele criará um novo recurso de entrada que tem os valores ao quadrado da entrada `x` (ou seja, grau=2).

In [ ]:
# Instanciar a classe para criar recursos polinomiais
poly = PolynomialFeatures(degree=2, include_bias=False)

# Calcular o número de recursos e transformar o conjunto de treinamento
X_train_mapped = poly.fit_transform(x_train)

# Visualize os primeiros 5 elementos do novo conjunto de treinamento. A coluna da esquerda é `x` e a da direita é `x^2`
# Observação: o `e+<número>` na saída indica quantas casas o ponto decimal deve 
# ser movido. Por exemplo, `3.24e+03` é igual a `3240`
print(X_train_mapped[:5])

Em seguida, você dimensionará as entradas como antes para restringir o intervalo de valores.

In [ ]:
# Instanciar a classe
scaler_poly = StandardScaler()

# Calcule a média e o desvio padrão do conjunto de treinamento e, em seguida, transforme-o
X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)

# Visualize os primeiros 5 elementos do conjunto de treinamento escalado.
print(X_train_mapped_scaled[:5])

Em seguida, você pode prosseguir com o treinamento do modelo. Depois disso, você medirá o desempenho do modelo em relação ao conjunto de validação cruzada. Como antes, você deve se certificar de realizar as mesmas transformações que fez no conjunto de treinamento. Você adicionará o mesmo número de recursos polinomiais e dimensionará o intervalo de valores.

In [ ]:
# Inicializar a classe
model = LinearRegression()

# Treinar o modelo
model.fit(X_train_mapped_scaled, y_train )

# Calcular o MSE de treinamento
yhat = model.predict(X_train_mapped_scaled)
print(f"MSE de treino: {mean_squared_error(y_train, yhat) / 2}")

# Adicionar os recursos polinomiais ao conjunto de validação cruzada
X_cv_mapped = poly.transform(x_cv)

# Dimensionar o conjunto de validação cruzada usando a média e o desvio padrão do conjunto de treinamento
X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)

# Calcular a validação cruzada MSE
yhat = model.predict(X_cv_mapped_scaled)
print(f"MSE de validação cruzada: {mean_squared_error(y_cv, yhat) / 2}")

Você notará que os MSEs são significativamente melhores tanto para o conjunto de treino quanto para o conjunto de validação cruzada quando você adicionou o polinômio de 2ª ordem. É possível que você queira introduzir mais termos polinomiais e ver qual deles apresenta o melhor desempenho. Conforme mostrado na aula, você pode ter 10 modelos diferentes como este:

<img src='images/C2_W3_poly.png' width=50%>

Você pode criar um loop que contenha todas as etapas das células de código anteriores. Aqui está uma implementação que adiciona recursos polinomiais de até grau = 10. Vamos plotá-la no final para facilitar a comparação dos resultados de cada modelo.

In [ ]:
# Inicializar listas para salvar os erros, os modelos e as transformações de recursos
train_mses = []
cv_mses = []
models = []
polys = []
scalers = []

# Faça um loop de 10 vezes. Cada vez adicionando mais um grau de polinômio maior que o anterior.
for degree in range(1,11):
    
    # Adicionar recursos polinomiais ao conjunto de treinamento
    poly = PolynomialFeatures(degree, include_bias=False)
    X_train_mapped = poly.fit_transform(x_train)
    polys.append(poly)
    
    # Escalar o conjunto de treinamento
    scaler_poly = StandardScaler()
    X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)
    scalers.append(scaler_poly)
    
    # Criar e treinar o modelo
    model = LinearRegression()
    model.fit(X_train_mapped_scaled, y_train )
    models.append(model)
    
    # Calcular o MSE de treinamento
    yhat = model.predict(X_train_mapped_scaled)
    train_mse = mean_squared_error(y_train, yhat) / 2
    train_mses.append(train_mse)
    
    # Adicionar recursos polinomiais e dimensionar o conjunto de validação cruzada
    X_cv_mapped = poly.transform(x_cv)
    X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)
    
    # Calcular a validação cruzada MSE
    yhat = model.predict(X_cv_mapped_scaled)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    cv_mses.append(cv_mse)
    
# Plotar os resultados
degrees=range(1,11)
utils.plot_train_cv_mses(degrees, train_mses, cv_mses, title="grau do polinômio vs. MSEs do trem e do CV")

### Escolhendo o melhor modelo

Ao selecionar um modelo, você deve escolher um que tenha bom desempenho tanto no conjunto de treinamento quanto no de validação cruzada. Isso significa que ele é capaz de aprender os padrões do conjunto de treinamento sem se ajustar demais. Se você usou os padrões neste laboratório, notará uma queda acentuada no erro de validação cruzada dos modelos com grau = 1 para grau = 2. Isso é seguido por uma linha relativamente plana até grau=5. Depois disso, no entanto, o erro de validação cruzada geralmente piora à medida que você adiciona mais recursos polinomiais. Diante disso, você pode decidir usar o modelo com o menor `cv_mse` como o mais adequado para o seu aplicativo.

In [ ]:
# Obtenha o modelo com o menor CV MSE (adicione 1 porque os índices da lista começam em 0)
# Isso também corresponde ao grau do polinômio adicionado
degree = np.argmin(cv_mses) + 1
print(f"O CV MSE mais baixo é encontrado no modelo com grau={degree}")

Em seguida, você pode imprmir o erro de generalização calculando o MSE do conjunto de teste. Como de costume, você deve transformar esses dados da mesma forma que fez com os conjuntos de treinamento e validação cruzada.

In [ ]:
# Adicionar recursos polinomiais ao conjunto de teste
X_test_mapped = polys[degree-1].transform(x_test)

# Escalar o conjunto de teste
X_test_mapped_scaled = scalers[degree-1].transform(X_test_mapped)

# Calcular o MSE de teste
yhat = models[degree-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y_test, yhat) / 2

print(f"MSE de treino: {train_mses[degree-1]:.2f}")
print(f"MSE de validação cruzada: {cv_mses[degree-1]:.2f}")
print(f"MSE de teste: {test_mse:.2f}")

## Redes neurais

O mesmo processo de seleção de modelos também pode ser usado ao escolher entre diferentes arquiteturas de redes neurais. Nesta seção, você criará os modelos mostrados abaixo e os aplicará à mesma tarefa de regressão acima.

<img src='images/C2_W3_NN_Arch.png' width=40%>

### Preparar os dados

Você usará os mesmos conjuntos de treinamento, validação cruzada e teste que gerou na seção anterior.

Com base em aulas anteriores, você já deve saber que as redes neurais podem aprender relações não lineares, portanto, pode optar por não adicionar recursos polinomiais. O código ainda está incluído abaixo, caso você queira tentar mais tarde e ver o efeito que isso terá nos resultados. O `degree` padrão é definido como `1` para indicar que ele usará apenas `x_train`, `x_cv` e `x_test` como estão (ou seja, sem nenhum recurso polinomial adicional).

In [ ]:
# Adicionar recursos polinomiais
degree = 1
poly = PolynomialFeatures(degree, include_bias=False)
X_train_mapped = poly.fit_transform(x_train)
X_cv_mapped = poly.transform(x_cv)
X_test_mapped = poly.transform(x_test)

Em seguida, você dimensionará os recursos de entrada para ajudar o gradiente de descida a convergir mais rapidamente. Novamente, observe que você está usando a média e o desvio padrão calculados a partir do conjunto de treinamento usando apenas `transform()` na validação cruzada e nos conjuntos de teste em vez de `fit_transform()`.

In [ ]:
# Escale as características usando o z-score
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train_mapped)
X_cv_mapped_scaled = scaler.transform(X_cv_mapped)
X_test_mapped_scaled = scaler.transform(X_test_mapped)

### Criar e treinar os modelos

Em seguida, você criará as arquiteturas de rede neural mostradas anteriormente. O código é fornecido na função `build_models()` no arquivo `utils.py`, caso você queira inspecioná-lo ou modificá-lo. Você o usará no loop abaixo e, em seguida, prosseguirá com o treinamento dos modelos. Para cada modelo, você também registrará os erros de treinamento e validação cruzada.

In [ ]:
# Inicializar as listas que conterão os erros de cada modelo
nn_train_mses = []
nn_cv_mses = []

# Criar os modelos
nn_models = utils.build_models()

# Fazer um loop sobre os modelos
for model in nn_models:
    
    # Configurar a perda e o otimizador
    model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    )

    print(f"Treinando {model.name}...")
    
    # Treinar o modelo
    model.fit(
        X_train_mapped_scaled, y_train,
        epochs=300,
        verbose=0
    )
    
    print("Concluído!\n")

    
    # Registre as MPEs de treinamento
    yhat = model.predict(X_train_mapped_scaled)
    train_mse = mean_squared_error(y_train, yhat) / 2
    nn_train_mses.append(train_mse)
    
    # Registre os MSEs da validação cruzada 
    yhat = model.predict(X_cv_mapped_scaled)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    nn_cv_mses.append(cv_mse)

    
# imprimir resultados
print("RESULTADOS:")
for model_num in range(len(nn_train_mses)):
    print(
        f"Modelo {model_num+1}: MSE de treino: {nn_train_mses[model_num]:.2f}, " +
        f"MSE de VC: {nn_cv_mses[model_num]:.2f}"
        )

Com base nos erros registrados, você pode decidir qual é o melhor modelo para o sua aplicação. Observe os resultados acima e veja se você concorda com o `model_num` selecionado abaixo. Por fim, você calculará o erro de teste para estimar o grau de generalização para novos exemplos.

In [ ]:
# Selecione o modelo com o menor CV MSE
model_num = 3

# Calcule o MSE de teste
yhat = nn_models[model_num-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y_test, yhat) / 2

print(f"Modelo Selecionado: {model_num}")
print(f"MSE de treino: {nn_train_mses[model_num-1]:.2f}")
print(f"MSE de validação cruzada: {nn_cv_mses[model_num-1]:.2f}")
print(f"MSE de teste: {test_mse:.2f}")

## Classificação

Nesta última parte do laboratório, você praticará a avaliação e a seleção de modelos em uma tarefa de classificação. O processo será semelhante, com a principal diferença sendo o cálculo dos erros. Você verá isso nas seções a seguir.

### Carregar o conjunto de dados

Primeiro, você carregará um conjunto de dados para uma tarefa de classificação binária. Ele tem 200 exemplos de dois recursos de entrada (`x1` e `x2`) e um alvo `y` de `0` ou `1`.

In [ ]:
# Carregar o conjunto de dados de um arquivo de texto
data = np.loadtxt('./data/data_w3_ex2.csv', delimiter=',')

# Dividir as entradas e saídas em matrizes separadas
x_bc = data[:,:-1]
y_bc = data[:,-1]

# Converta y em 2-D porque os comandos posteriores exigirão isso (x já é 2-D)
y_bc = np.expand_dims(y_bc, axis=1)

print(f"o formato das entradas x é: {x_bc.shape}")
print(f" formato dos alvos y é: {y_bc.shape}")

Você pode plotar o conjunto de dados para examinar como os exemplos são separados.

In [ ]:
utils.plot_bc_dataset(x=x_bc, y=y_bc, title="x1 vs. x2")

### Dividir e preparar o conjunto de dados

Em seguida, você gerará os conjuntos de treinamento, validação cruzada e teste. Você usará as mesmas proporções 60/20/20 de antes. Você também dimensionará os recursos como fez na seção anterior.

In [ ]:
from sklearn.model_selection import train_test_split

# Obtenha 60% do conjunto de dados como o conjunto de treinamento.
# Coloque os 40% restantes em variáveis temporárias.
x_bc_train, x_, y_bc_train, y_ = train_test_split(x_bc, y_bc, test_size=0.40, random_state=1)

# Dividir o subconjunto de 40% acima em dois: uma metade para validação cruzada e a outra para o conjunto de teste
x_bc_cv, x_bc_test, y_bc_cv, y_bc_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Excluir variáveis temporárias
del x_, y_

print(f"o formato do conjunto de treinamento (entrada) é: {x_bc_train.shape}")
print(f"o formato do conjunto de treinamento (alvo) é: {y_bc_train.shape}\n")
print(f"o formato do conjunto de validação cruzada (entrada) é: {x_bc_cv.shape}")
print(f"o formato do conjunto de validação cruzada (alvo) é: {y_bc_cv.shape}\n")
print(f"o formato do conjunto de teste (entrada) é: {x_bc_test.shape}")
print(f"o formato do conjunto de teste (alvo) é: {y_bc_test.shape}")

In [ ]:
# Escalar os recursos

# Inicializar a classe
scaler_linear = StandardScaler()

# Calcule a média e o desvio padrão do conjunto de treinamento e, em seguida, transforme-o
x_bc_train_scaled = scaler_linear.fit_transform(x_bc_train)
x_bc_cv_scaled = scaler_linear.transform(x_bc_cv)
x_bc_test_scaled = scaler_linear.transform(x_bc_test)

### Avaliando o erro dos modelos de classificação

Nas seções anteriores sobre modelos de regressão, você usou o erro quadrático médio para medir o desempenho do seu modelo. Para classificação, você pode obter uma métrica semelhante obtendo a fração dos dados que o modelo classificou incorretamente. Por exemplo, se o seu modelo fez previsões erradas para 2 amostras de 5, você informará um erro de `40%` ou `0,4`. O código abaixo demonstra isso usando um loop for e também com a função [`mean()`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) do Numpy. 

In [ ]:
# Saída do modelo de amostra
probabilities = np.array([0.2, 0.6, 0.7, 0.3, 0.8])

# Aplique um limite à saída do modelo. Se for maior que 0,5, defina como 1. Caso contrário, 0.
predictions = np.where(probabilities >= 0.5, 1, 0)

# Rótulos de alvo (ground-truth)
ground_truth = np.array([1, 1, 1, 1, 1])

# Inicializar o contador de dados com classificação incorreta
misclassified = 0

# Obter o número de previsões
num_predictions = len(predictions)

# Fazer um loop em cada previsão
for i in range(num_predictions):
    
    # Verificar se corresponde ao ground-truth
    if predictions[i] != ground_truth[i]:
        
        # Adicione um ao contador se a previsão estiver errada
        misclassified += 1

# Calcule a fração dos dados que o modelo classificou incorretamente
fraction_error = misclassified/num_predictions

print(f"probabilidades: {probabilities}")
print(f"predições com limiar=0.5: {predictions}")
print(f"alvos: {ground_truth}")
print(f"fração de dados com classificação incorreta (loop-for): {fraction_error}")
print(f"fração de dados com classificação incorreta (com np.mean()): {np.mean(predictions != ground_truth)}")

### Construir e treinar o modelo

Você usará as mesmas arquiteturas de rede neural da seção anterior, portanto, poderá chamar a função `build_models()` novamente para criar novas instâncias desses modelos. 

Você seguirá a abordagem recomendada mencionada na semana passada, em que usa uma ativação `linear` para a camada de saída (em vez de `sigmoid`) e define `from_logits=True` ao declarar a função de perda do modelo. Você usará a [binary crossentropy loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy) porque esse é um problema de classificação binária.

Após o treinamento, você usará uma [sigmoid function](https://www.tensorflow.org/api_docs/python/tf/math/sigmoid) para converter os resultados do modelo em probabilidades. A partir daí, você pode definir um limite e obter a fração de exemplos com classificação incorreta dos conjuntos de treinamento e validação cruzada.

Você pode ver tudo isso na célula de código abaixo.

In [ ]:
# Inicializar as listas que conterão os erros de cada modelo
nn_train_error = []
nn_cv_error = []

# Criar os modelos
models_bc = utils.build_models()

# Fazer um loop em cada modelo
for model in models_bc:
    
    # Configurar a perda e o otimizador
    model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    )

    print(f"Treinando {model.name}...")

    # Treinar o modelo
    model.fit(
        x_bc_train_scaled, y_bc_train,
        epochs=200,
        verbose=0
    )
    
    print("Done!\n")
    
    # Definir o limite para classificação
    threshold = 0.5
    
    # Registre a fração de exemplos com classificação incorreta para o conjunto de treinamento
    yhat = model.predict(x_bc_train_scaled)
    yhat = tf.math.sigmoid(yhat)
    yhat = np.where(yhat >= threshold, 1, 0)
    train_error = np.mean(yhat != y_bc_train)
    nn_train_error.append(train_error)

    # Registre a fração de exemplos com classificação incorreta para o conjunto de validação cruzada
    yhat = model.predict(x_bc_cv_scaled)
    yhat = tf.math.sigmoid(yhat)
    yhat = np.where(yhat >= threshold, 1, 0)
    cv_error = np.mean(yhat != y_bc_cv)
    nn_cv_error.append(cv_error)

# Imprimir o resultado
for model_num in range(len(nn_train_error)):
    print(
        f"Modelo {model_num+1}: Erro de classificação do conjunto de treinamento: {nn_train_error[model_num]:.5f}, " +
        f"Erro de classificação do conjunto VC: {nn_cv_error[model_num]:.5f}"
        )

Com base no resultado acima, você pode escolher qual deles teve o melhor desempenho. Se houver um empate no erro do conjunto de validação cruzada, você poderá adicionar outro critério para desempatar. Por exemplo, você pode escolher o que tiver um erro de treinamento menor. Uma abordagem mais comum é escolher o modelo menor porque ele economiza recursos computacionais. Em nosso exemplo, o Modelo 1 é o menor e o Modelo 3 é o maior.

Por fim, você pode calcular o erro de teste para informar o erro de generalização do modelo.

In [ ]:
# Selecione o modelo com o menor erro
model_num = 3

# Calcular o erro de teste
yhat = models_bc[model_num-1].predict(x_bc_test_scaled)
yhat = tf.math.sigmoid(yhat)
yhat = np.where(yhat >= threshold, 1, 0)
nn_test_error = np.mean(yhat != y_bc_test)

print(f"Modelo Selecionado: {model_num}")
print(f"Erro de classificação do conjunto de treinamento: {nn_train_error[model_num-1]:.4f}")
print(f"Erro de classificação do conjunto VC: {nn_cv_error[model_num-1]:.4f}")
print(f"Erro de classificação do conjunto de teste: {nn_test_error:.4f}")

## Introdução ao [Keras Tuner](https://www.tensorflow.org/tutorials/keras/keras_tuner)

## Ajuste de hiperparâmetros com o [painel HParams](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams)

## Resumo

Neste laboratório, você praticou a avaliação do desempenho de um modelo e a escolha entre diferentes configurações de modelos. Você dividiu seus conjuntos de dados em conjuntos de treinamento, validação cruzada e teste e viu como cada um deles é usado em aplicativos de aprendizado de máquina. Na próxima seção do curso, você verá mais dicas sobre como aprimorar seus modelos diagnosticando o viés e a variação. Continue assim!